In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "qauestion": "The Lion King",
        "emojis": "🦁👑🌅",
    },
    {
        "qauestion": "Titanic",
        "emojis": "🚢💔🌊",
    },
    {
        "qauestion": "Top Gun",
        "emojis": "🛩️👨‍✈️🔥",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{qauestion}"),
        ("ai", "{emojis}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie emoji expert. You respond to movie titles with exactly three relevant emojis."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("Inception")

content='🧠🌀🚪'


In [19]:
invoke_chain("Avatar")

content='🌿🌌💙'


In [20]:
invoke_chain("처음 물어본 영화의 제목이 뭐였나요?")

content='Inception'
